In [6]:
import spider as sp
import scanpy as sc
import numpy as np

In [11]:
real_adata = sc.read_h5ad("../data/STARmap_15ct/15ct_realdata.h5ad")

## estimate parameters from the reference data

In [16]:
def get_trans(adata = None, ct = None, neighbors = None):
    sn = sp.get_spaital_network(Num_sample=adata.obs.shape[0],
                         spatial=adata.obsm["spatial"], coord_type = "generic",
                         n_neighs=neighbors)
    onehot_ct = sp.get_onehot_ct(init_assign=ct)
    nb_count = np.array(sn * onehot_ct, dtype=np.float32)
    target_trans = sp.get_nb_freq(nb_count=nb_count, onehot_ct=onehot_ct)
    return target_trans

In [17]:
Num_ct_sample = np.bincount(real_adata.obs.label)

prior  = Num_ct_sample/np.sum(Num_ct_sample)

real_celltype = np.unique(real_adata.obs.celltype)

match_list = dict(zip(range(len(Num_ct_sample)),real_celltype))

target_trans = get_trans(adata = real_adata , ct = real_adata.obs.label,neighbors = 8)

/Users/era/anaconda3/envs/spider/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## simulated data using Spider

In [19]:
np.random.seed(1)
res, cell_location = sp.simulate_10X(cell_num=None,
                        Num_celltype=None,
                        prior=None,
                        target_trans=target_trans,
                        spot_radius=None,
                        spot_min=None,
                        spot_max=None,
                        image_width=None,
                        image_height=None,
                        cell_location=None,
                        tol=2e-2,
                        T=1e-2,
                        loop_times=None,
                        smallsample_max_iter=100000,
                        bigsample_max_iter=10000,
                        ref=real_adata)

/Users/era/anaconda3/envs/spider/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/era/anaconda3/envs/spider/lib/python3.9/site-packages/spider/Annealing.py:81: RuntimeWarning: invalid value encountered in double_scalars
  if 0 > (dist_before - dist_after):


swap_num:  1
Refine cell type using Metropolis–Hastings algorithm.
Sample num:1523
 1000 iteration, error 1.416
 2000 iteration, error 1.368
 3000 iteration, error 1.433
 4000 iteration, error 1.438
 5000 iteration, error 1.398
 6000 iteration, error 1.374
 7000 iteration, error 1.398
 8000 iteration, error 1.381
 9000 iteration, error 1.373
10000 iteration, error 1.437
11000 iteration, error 1.371
12000 iteration, error 1.376
13000 iteration, error 1.335
14000 iteration, error 1.210
15000 iteration, error 1.135
16000 iteration, error 1.072
17000 iteration, error 1.021
18000 iteration, error 0.991
19000 iteration, error 0.938
20000 iteration, error 0.927
21000 iteration, error 0.885
22000 iteration, error 0.881
23000 iteration, error 0.818
24000 iteration, error 0.814
25000 iteration, error 0.801
26000 iteration, error 0.787
27000 iteration, error 0.777
28000 iteration, error 0.760
29000 iteration, error 0.746
30000 iteration, error 0.747
31000 iteration, error 0.727
32000 iteration, e

In [21]:

def get_sim_cell_level_expr(celltype_assignment = None, adata = None,
                            Num_celltype = None, Num_ct_sample = None,
                            match_list = None, ct_key = None):
    ## 给每一个估计的细胞，分配一种单细胞里面的细胞index，
    ## 前提是同一组细胞的index对应到单细胞数据里面是同一种细胞类型
    assert adata is not None, print("please input an adata about singlecell, if you don't want to find it in your folder,"+
               "here we We provide a single-cell data simulation strategy for you to choose")
        
    
    idx_list = celltype_assignment.copy()
    for i in range(Num_celltype):
        ## 找到单细胞中的指定细胞类型的细胞
        ct_idx = np.where(adata.obs[ct_key]==match_list[i])[0]
        ## 抽取指定模拟该细胞类型数量的细胞，得到在单细胞里面的idx
        sim_cell_idx = np.random.choice(ct_idx,size = Num_ct_sample[i],replace = True)

        idx_list[np.where(celltype_assignment==i)[0]] = sim_cell_idx

    sim_cell_expr = adata[idx_list]
    #sim_cell_expr.obs["sim_celltype"] = res  
    return sim_cell_expr

In [23]:
spider_adata = get_sim_cell_level_expr(celltype_assignment=res,
                                       adata=real_adata,
                                       Num_celltype=len(Num_ct_sample),
                                       Num_ct_sample=Num_ct_sample,
                                       match_list=match_list,
                                       ct_key="celltype")
spider_adata.obsm["spatial"] = real_adata.obsm["spatial"]
spider_adata.obs.index = [
    'cell' + str(j) for j in range(spider_adata.shape[0])
]

/var/folders/_s/mg8jr2xx3_vg8zqjqwyjv0_m0000gn/T/ipykernel_26542/1495643098.py:7: ImplicitModificationWarning: Setting element `.obsm['spatial']` of view, initializing view as actual.
  spider_adata.obsm["spatial"] = real_adata.obsm["spatial"]
/Users/era/anaconda3/envs/spider/lib/python3.9/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [24]:
spider_adata

AnnData object with n_obs × n_vars = 1523 × 882
    obs: 'celltype', 'label'
    var: 'gene'
    uns: 'svg_scanpy', 'svg_squidpy'
    obsm: 'spatial'